Notes on draft position statistics:

-The CurrentRoster only contains for players on the 2018 roster right now. I actually don't have the ability to generate each year's rosters going back in time, ESPN doesn't make that data available anymore, which is really frustrating.

-After inheriting Cody's team, a team which missed the 2015 draft altogether, Jon was allowed to pick up any number of free agents. He "drafted" 10 of them, and they count in the 10th round of the 2015 draft.

-Counts the most recent time the player was drafted into our league, even if not by the player's current team (in the case of trades for example).

Average Draft Year for each roster.

Todd has the oldest roster by year they were drafted, while Brian and William are using more recently drafted players.

In [3]:
select Current_owner, Avg_Draft_Year = round(avg(draftyear*1.),2) from CurrentRoster 
where rosteryear = 2018
Group by Current_Owner order by Avg_Draft_Year;

(8 rows affected)

Total execution time: 00:00:00.0789448

Current_owner,Avg_Draft_Year
Todd,2015.760000
Paul,2015.960000
Adrian,2015.960000
Ian,2016.000000
Christine,2016.080000
Jonathan,2016.120000
Brian,2016.200000
William,2016.200000


Average Draft Round for each roster

In 2018, Brian used, on average, players that were drafted two rounds higher than Adrian.

In [4]:
select Current_owner, Avg_Draft_Round = round(avg(draft_round*1.),2) from CurrentRoster 
where rosteryear = 2018
Group by Current_Owner order by Avg_Draft_Round;

(8 rows affected)

Total execution time: 00:00:00.0727022

Current_owner,Avg_Draft_Round
Brian,4.200000
Paul,4.440000
Christine,4.640000
William,4.920000
Todd,4.960000
Jonathan,5.560000
Ian,6.160000
Adrian,6.480000


Average number of draft picks pear year 

-Includes Jon's "10th round" of picks after he inherited Cody's lost draft of 2015

-Remember in the first two seasons, we only picked 8 players in the draft. 

-Counts any 10th round picks used for the "Injured After Drops" rule

In [5]:
with cteDraftPicksperYear (teamid, draftyear, numberofpicks) as (select teamid, draftyear, count(id) from dbo.draftresults group by teamid, draftyear)
select t.TeamOwner, avg_numberofpicks = round(avg(numberofpicks*1.),3) 
from cteDraftPicksperYear dr
inner join team t on t.id = dr.TeamID 
where dr.draftyear > 2012 --don't count our 25-round initial seed draft
group by t.TeamOwner
order by avg_numberofpicks

(10 rows affected)

Total execution time: 00:00:00.0732812

TeamOwner,avg_numberofpicks
Michael,8.500000
Patrick,8.500000
Todd,8.500000
William,8.833000
Paul,8.833000
Brian,8.833000
Christine,8.833000
Ian,9.167000
Adrian,9.250000
Jonathan,11.500000


Average number of draft picks pear year 

-Remember in the first two seasons, we only picked 8 players in the draft. 

-Only includes the first 9 rounds of picks, so this average is only affected by trades

In [6]:
with cteDraftPicksperYear (teamid, draftyear, numberofpicks) as (select teamid, draftyear, count(id) from dbo.draftresults where draft_round <= 9 group by teamid, draftyear)
select t.TeamOwner, avg_numberofpicks = round(avg(numberofpicks*1.),3) 
from cteDraftPicksperYear dr
inner join team t on t.id = dr.TeamID 
where dr.draftyear > 2012 --don't count our 25-round initial seed draft
group by t.TeamOwner
order by avg_numberofpicks

(10 rows affected)

Total execution time: 00:00:00.0609150

TeamOwner,avg_numberofpicks
Michael,8.500000
Patrick,8.500000
Todd,8.500000
William,8.833000
Paul,8.833000
Brian,8.833000
Christine,8.833000
Adrian,9.000000
Jonathan,9.000000
Ian,9.167000


Average pick in the round

-Mostly determined by pre-draft order, but could be affected by:

-Trades

-The  draft party mock draft game

In [7]:
select Current_owner, Avg_pick_in_draft_round = round(avg(pick_in_draft_round*1.),2) from CurrentRoster 
where rosteryear = 2018
group by Current_Owner 
order by Avg_pick_in_draft_round;

(8 rows affected)

Total execution time: 00:00:00.0732824

Current_owner,Avg_pick_in_draft_round
Jonathan,3.200000
Brian,3.440000
Ian,3.880000
William,4.160000
Todd,4.840000
Paul,4.960000
Adrian,5.040000
Christine,5.720000


Average draft round by position drafted. Who drafts QB's high? 

-William drafts RB's high, not that it's done him any good. 

-Adrian drafts QB's the latest, and yet he's still stacked with QB's right now.

-Todd drafts kickers slightly earlier than everyone else

-Brian loves to take WR's early

-Only includes the first 9 rounds

In [8]:
select TeamOwner, RB, TE, WR, QB, K from (
select t.TeamOwner, dr.position, Avg_draft_round = cast(round(avg(draft_round*1.),2)as decimal(9,2)) from draftresults  dr
inner join team t on t.id = dr.TeamID 
where draft_round <= 9 
and t.TeamOwner not in ('Michael','Patrick')
group by t.TeamOwner, dr.Position
) x 
PIVOT (AVG(Avg_draft_round) for Position in (RB, TE, WR, QB, K)) as pvt
ORDER BY RB;

(10 rows affected)

Total execution time: 00:00:00.0738784

TeamOwner,RB,TE,WR,QB,K
William,2.720000,6.000000,4.380000,6.270000,7.140000
Paul,4.210000,6.000000,4.500000,5.000000,7.200000
Todd,4.260000,5.270000,4.330000,4.000000,6.000000
Michael,4.330000,4.000000,4.670000,6.000000,6.330000
Patrick,4.380000,7.000000,5.380000,2.500000,NULL
Jonathan,4.620000,4.830000,6.000000,4.670000,6.750000
Christine,4.650000,5.250000,5.000000,4.440000,7.200000
Ian,4.900000,4.440000,5.050000,4.330000,6.800000
Adrian,5.170000,3.670000,4.560000,6.500000,6.330000
Brian,5.710000,5.450000,3.590000,3.900000,7.500000


More on what positions people draft, and when.

-William drafts RB's much higher than everyone else, consistently, and drafts QB's later than everyone else.

-Brian drafts WR's a round higher than everyone else.

-Todd drafts a TE in the 5th-6th rounds, consistently.

-Christine is least predictable when it comes to drafting.

In [9]:
select t.TeamOwner, dr.position
, Avg_draft_round = round(avg(draft_round*1.),2)
, count_of_picks_of_this_position = count(dr.id) 
, STDEV_draft_round = cast(STDEV(draft_round*1.) as decimal(9,2)) 
from draftresults  dr
inner join team t on t.id = dr.TeamID 
where draft_round <= 9 
and t.TeamOwner not in ('Michael','Patrick')
and position <>'K'
group by t.TeamOwner, dr.Position
having count(dr.id)  > 3 -- not enough sample size
ORDER BY Avg_Draft_round;

(35 rows affected)

Total execution time: 00:00:00.0711841

TeamOwner,position,Avg_draft_round,count_of_picks_of_this_position,STDEV_draft_round
William,RB,2.720000,18,1.90
Brian,WR,3.590000,17,2.21
Brian,QB,3.900000,10,2.08
Todd,QB,4.000000,9,2.74
Michael,TE,4.000000,4,2.16
Paul,RB,4.210000,19,2.70
Todd,RB,4.260000,19,2.60
Michael,RB,4.330000,9,2.55
Ian,QB,4.330000,9,2.87
Todd,WR,4.330000,12,3.03


Number of players at each position drafted + average number of that position picked per draft

-Not everyone has participated in the same number of drafts. Jon and Adrian have only been in the league 4 seasons, not all 7.

-Christine has drafted more WR's than anyone, but Adrian is catching up, drafting an avg of 4 per year.

-William  has drafted the most QB's, a lot of good it's done.

-Brian has drafted the most kickers

-Todd is super skeptical of WR's, drafting less than 2 per draft, far fewer than everyone else.

In [13]:
WITH cteCount (TeamOwner, position, count_of_picks_of_this_position, count_of_drafts) as (
 select t.TeamOwner, dr.position
, count_of_picks_of_this_position = count(dr.id)  
, count_of_drafts = count(distinct dr.draftyear)   
from draftresults  dr
inner join team t on t.id = dr.TeamID 
where draft_round <= 9 
and t.TeamOwner not in ('Michael','Patrick')
group by t.TeamOwner, dr.position
) 
SELECT TeamOwner, WR, RB, QB, TE, K FROM (
select teamowner, position
,  Count_of_picks_Avg_per_draft = cast(count_of_picks_of_this_position  as varchar(4)) + ' ' +  cast(cast(count_of_picks_of_this_position*1./ MAX(count_of_drafts*1.) OVER (PARTITION BY TeamOwner) as decimal(9,2)) as varchar(6))
from cteCount 
) x
PIVOT ( MAX(Count_of_picks_Avg_per_draft) FOR Position IN (WR, RB, QB, TE, K)) AS pvt
ORDER BY WR desc;

(8 rows affected)

Total execution time: 00:00:00.0588292

TeamOwner,WR,RB,QB,TE,K
Christine,23 3.29,17 2.43,9 1.29,8 1.14,5 0.71
Ian,20 2.86,21 3.00,9 1.29,9 1.29,5 0.71
Paul,20 2.86,19 2.71,8 1.14,10 1.43,5 0.71
Brian,17 2.43,14 2.00,10 1.43,11 1.57,10 1.43
Adrian,16 4.00,12 3.00,2 0.50,3 0.75,3 0.75
William,16 2.29,18 2.57,11 1.57,10 1.43,7 1.00
Todd,12 1.71,19 2.71,9 1.29,11 1.57,9 1.29
Jonathan,10 2.50,13 3.25,3 0.75,6 1.50,4 1.00
